In [ ]:
# Change working directory to the project root
import os
os.chdir('..')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from lib.feature import *

In [3]:
MATCH_RATE_THRESHOLD = 0.4

In [4]:
df_tlist = pd.read_csv('data/meta/tracklist.csv')
df_tlist.head()

,i_track,track_id,timestamp,artist,title,filename
0,0,0tNDAyqBm2A,0,Sumbawanga,Yebo,00-0tNDAyqBm2A.wav
1,1,yAFEClw71TM,320,Raf Dee-Jay,Hey Girl (Club Mix),01-yAFEClw71TM.wav
2,2,JtPpF2K8G3g,516,Jam Tribe,Dozal,02-JtPpF2K8G3g.wav
3,3,Ri2xF0NYLI4,650,Bailoo,Tip and Touch (Come On!) (Large Mello Mix),03-Ri2xF0NYLI4.wav
4,4,j6NZ-6zow60,795,Pierre Akendengue,Epuguzu,04-j6NZ-6zow60.wav


In [5]:
df_align = pd.concat([pd.read_pickle(p) for p in glob('data/align/*.pkl')])
df_align = df_align[df_align.match_rate > MATCH_RATE_THRESHOLD]
df_align = df_align[(df_align.feature == 'chroma+mfcc') & df_align.key_invariant]
df_align.head()

,mix_id,track_id,feature,key_invariant,match_rate,key_change,best_cost,costs,wp,mix_cue_in_time,mix_cue_out_time,track_cue_in_time,track_cue_out_time,mix_cue_in_beat,mix_cue_out_beat,track_cue_in_beat,track_cue_out_beat
0,cPo-qzbGLqE,0tNDAyqBm2A,chroma+mfcc,True,0.819165,0,2.006086,"[2.0060858438369937, 3.058442253291467, 3.1677...","[[647, 555], [646, 555], [645, 555], [644, 555...",7.48,283.88,1.14,292.53,26,534,2,518
4,cPo-qzbGLqE,j6NZ-6zow60,chroma+mfcc,True,0.581818,0,2.371457,"[2.371456746217164, 3.1854158131108856, 3.2904...","[[440, 1649], [439, 1649], [438, 1649], [437, ...",753.62,884.48,6.44,142.29,1396,1636,11,251
5,cPo-qzbGLqE,RocnNoPCZDw,chroma+mfcc,True,0.790281,0,2.300925,"[2.300924634703462, 3.2198080656182695, 3.3423...","[[391, 1962], [390, 1962], [389, 1962], [388, ...",893.74,1060.07,7.78,190.75,1653,1958,13,318
6,cPo-qzbGLqE,urJv3U_fgCY,chroma+mfcc,True,0.659696,0,2.396316,"[2.3963156009612945, 3.3051269882962155, 3.490...","[[526, 2316], [525, 2315], [524, 2315], [523, ...",1067.16,1246.87,15.03,202.38,1971,2302,26,357
7,cPo-qzbGLqE,_Y8rjKHE3Ew,chroma+mfcc,True,0.423831,0,2.795153,"[2.7951533998854856, 3.204821398828105, 3.2712...","[[663, 2636], [662, 2636], [661, 2635], [660, ...",1298.74,1423.00,187.50,312.95,2398,2628,412,642


In [6]:
df = df_tlist.merge(df_align)
df.shape

(23, 22)

In [7]:
mix_path = f'data/mix/{df.mix_id[0]}.wav'
mix_beats = beats(mix_path)

In [16]:
data = []
print(' # original_bpm adjusted_bpm bpm_change title')
for _, track in df.iterrows():
    track_path = f'data/track/{track.filename}'
    track_beats = beats(track_path)
    num_track_beats = track.track_cue_out_beat - track.track_cue_in_beat
    len_track = track_beats[track.track_cue_out_beat] - track_beats[track.track_cue_in_beat]
    track_bpm = num_track_beats / len_track * 60

    num_mix_beats = track.mix_cue_out_beat - track.mix_cue_in_beat
    len_mix = mix_beats[track.mix_cue_out_beat] - mix_beats[track.mix_cue_in_beat]
    mix_bpm = num_mix_beats / len_mix * 60

    bpm_change_ratio = mix_bpm / track_bpm
    
    print(f'{track.i_track:2}', f'{mix_bpm:12.1f}', f'{track_bpm:12.1f}', f'{bpm_change_ratio - 1:10.1%}', track.title)

 # original_bpm adjusted_bpm bpm_change title
 0        110.3        106.2       3.8% Yebo
 4        110.0        106.0       3.8% Epuguzu
 5        110.0        100.0      10.0% Pa Enerve Mwen
 6        110.5        106.0       4.3% Hailwa Yange Oike Mbela
 7        111.1        110.0       1.0% Ascona
 8        112.0        111.0       0.9% Tengo La Sangre Caliente (Base)
 9        111.7        108.0       3.5% Work It Out
10        112.0        110.3       1.5% Defected
13        117.8        118.4      -0.5% I Feel Good (Dub Version)
14        119.6        119.1       0.4% It's Too Late (Club Mix)
16        121.2        119.7       1.2% Deep Love (Slammin’ Da Jazz Mix)
17        122.2        120.5       1.4% Lately
18        123.5        123.3       0.2% Dance To The Music (HNRG Houzz Mix)
19        124.4        120.5       3.2% Everybody Up (Dub)
20        124.5        120.0       3.7% Beach Of The War Goddess (Tactical Instrumental)
21        124.6        123.0       1.3% Brother